<a href="https://colab.research.google.com/github/gsdos1984-sudo/EPPsimulatorby-Miguel-verastegui/blob/main/Epp_simulator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
%%writefile streamlit_app.py
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import xlsxwriter

st.title("Simulador EPP - Miguel Verástegui")



# ========================== COLAB: TODO EN UNA SOLA CELDA (con Drive) ==========================
# 0) Montar Google Drive (para guardar/leer archivos si quieres)
from google.colab import drive
drive.mount('/content/drive')

# 1) Instalar dependencias y LocalTunnel (para exponer la app)
!pip -q install streamlit pandas numpy matplotlib openpyxl > /dev/null
!npm -s install -g localtunnel > /dev/null

# 2) Escribir el archivo app.py con el simulador completo
%%writefile app.py
import os
from io import BytesIO
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import streamlit as st

# =========================================
# CONFIGURACIÓN INICIAL
# =========================================
st.set_page_config(page_title="Simulador EPP – Modelos + Wet/Dry", layout="wide", page_icon="🧪")
st.title("🧪 Simulador de Moldeo EPP – Modelos + Wet/Dry")

st.markdown("""
Este simulador permite:
1) Cargar tu **Excel** de modelos y elegir una pieza (Part Number / Part Name).
2) Mostrar **Bead**, **Cure Time** y objetivos **Wet/Dry** nominales del Excel.
3) Ajustar parámetros de proceso (ICP, vapor, filling, cooling, aging) para obtener **simulación** de **Wet/Dry** y **dimensiones**.
4) Ingresar tus **mediciones reales** y comparar: **SIM vs NOM vs REAL**.
5) Exportar un **reporte** a Excel (openpyxl) y opcionalmente **guardarlo en Drive**.
""")

# =========================================
# SIDEBAR: CARGA DE EXCEL
# =========================================
st.sidebar.header("📄 Archivo de modelos")
st.sidebar.write("Sube tu Excel `EPP PARTS PRD-WI-023 RDC Part and Weight ALL Model Rev 86.xlsx`")
up = st.sidebar.file_uploader("Excel de modelos", type=["xlsx"])

# Lectura flexible (múltiples hojas)
def normaliza_col(s: str) -> str:
    return (
        str(s).strip()
        .lower()
        .replace("\n"," ")
        .replace("\r"," ")
        .replace("\t"," ")
        .replace("  "," ")
    )

df_models = None
if up is not None:
    try:
        xl = pd.ExcelFile(up)
        frames = []
        for name in xl.sheet_names:
            try:
                tmp = xl.parse(name)
                if len(tmp.columns) == 0 or len(tmp) == 0:
                    continue
                tmp.columns = [normaliza_col(c) for c in tmp.columns]
                frames.append(tmp)
            except Exception:
                pass
        if frames:
            df_all = pd.concat(frames, ignore_index=True)

            # columnas candidatas
            cand_partnum = [c for c in df_all.columns if ("part" in c and "number" in c) or c in ("partnumber","part no.","p/n","pn")]
            cand_partname = [c for c in df_all.columns if ("part" in c and "name" in c) or c in ("description","desc")]
            cand_bead = [c for c in df_all.columns if "bead" in c or "density" in c or "material" in c]
            cand_cure = [c for c in df_all.columns if ("cure" in c and "time" in c) or "autocl" in c or "aging" in c]

            # Wet/Dry targets
            cand_wet_min = [c for c in df_all.columns if "wet" in c and "min" in c]
            cand_wet_nom = [c for c in df_all.columns if "wet" in c and ("nom" in c or "target" in c)]
            cand_wet_max = [c for c in df_all.columns if "wet" in c and "max" in c]

            cand_dry_min = [c for c in df_all.columns if "dry" in c and "min" in c]
            cand_dry_nom = [c for c in df_all.columns if "dry" in c and ("nom" in c or "target" in c)]
            cand_dry_max = [c for c in df_all.columns if "dry" in c and "max" in c]

            # Mapear columnas
            col_partnum = cand_partnum[0] if cand_partnum else None
            col_partname = cand_partname[0] if cand_partname else None
            col_bead = cand_bead[0] if cand_bead else None
            col_cure = cand_cure[0] if cand_cure else None

            col_wet_min = cand_wet_min[0] if cand_wet_min else None
            col_wet_nom = cand_wet_nom[0] if cand_wet_nom else None
            col_wet_max = cand_wet_max[0] if cand_wet_max else None

            col_dry_min = cand_dry_min[0] if cand_dry_min else None
            col_dry_nom = cand_dry_nom[0] if cand_dry_nom else None
            col_dry_max = cand_dry_max[0] if cand_dry_max else None

            keep_cols = []
            for c in [col_partnum, col_partname, col_bead, col_cure,
                      col_wet_min, col_wet_nom, col_wet_max,
                      col_dry_min, col_dry_nom, col_dry_max]:
                if c and c in df_all.columns:
                    keep_cols.append(c)
            df_models = df_all[keep_cols].dropna(how="all").drop_duplicates().reset_index(drop=True)

            # Renombrar de forma estable
            rename_map = {}
            if col_partnum: rename_map[col_partnum] = "part_number"
            if col_partname: rename_map[col_partname] = "part_name"
            if col_bead: rename_map[col_bead] = "bead_text"
            if col_cure: rename_map[col_cure] = "cure_time"
            if col_wet_min: rename_map[col_wet_min] = "wet_min"
            if col_wet_nom: rename_map[col_wet_nom] = "wet_nom"
            if col_wet_max: rename_map[col_wet_max] = "wet_max"
            if col_dry_min: rename_map[col_dry_min] = "dry_min"
            if col_dry_nom: rename_map[col_dry_nom] = "dry_nom"
            if col_dry_max: rename_map[col_dry_max] = "dry_max"
            df_models = df_models.rename(columns=rename_map)
    except Exception as e:
        st.error(f"Error leyendo el Excel: {e}")

if df_models is None:
    st.info("🔼 Sube el Excel para habilitar el selector de modelo.")
else:
    # Selector de modelo
    def etiqueta(row):
        pn = str(row.get("part_number","")).strip()
        nm = str(row.get("part_name","")).strip()
        return (pn + " — " + nm).strip(" —")

    opciones = df_models.apply(etiqueta, axis=1).tolist()
    sel = st.selectbox("🧾 Modelo", opciones)
    idx = opciones.index(sel) if sel in opciones else None

# =========================================
# Parámetros del PROCESO (heurístico EPP)
# =========================================
st.sidebar.header("⚙️ Parámetros de proceso")
L_base = st.sidebar.number_input("Largo nominal (mm)", 200, 3000, 1200, 10)
W_base = st.sidebar.number_input("Ancho nominal (mm)", 200, 3000, 800, 10)
T_base = st.sidebar.number_input("Espesor nominal (mm)", 10, 400, 50, 1)

steam_pressure = st.sidebar.slider("ICP / Presión de vapor (bar)", 0.6, 2.2, 1.5, 0.05)
steam_time = st.sidebar.slider("Tiempo de vapor (s)", 1, 20, 7, 1)
temp_fixed = st.sidebar.slider("Temp FIXED SIDE (°C)", 25, 110, 60, 1)
temp_mobile = st.sidebar.slider("Temp MOBILE SIDE (°C)", 25, 110, 50, 1)

fill_time1 = st.sidebar.slider("Filling time 1 (s)", 1, 10, 3, 1)
fill_time2 = st.sidebar.slider("Filling time 2 (s)", 0, 10, 2, 1)

cooling_time = st.sidebar.slider("Cooling time (s)", 3, 60, 20, 1)
water_on = st.sidebar.checkbox("Agua de enfriamiento ON", True)
aging_quality = st.sidebar.slider("Calidad de aging/autoclave (0=deficiente, 1=óptima)", 0.0, 1.0, 0.7, 0.05)

# Beads típicos EPP (densidad g/L nominal orientativa)
BEAD_DENSITIES = {15:50, 22:30, 35:20, 42:16}

def infer_bead_from_text(txt):
    if not isinstance(txt, str):
        return None
    t = txt.lower()
    for k in (15,22,35,42):
        if str(k) in t: return k
    for k,v in BEAD_DENSITIES.items():
        if f"{v}" in t: return k
    return None

# =========================================
# LÓGICA HEURÍSTICA (simulación)
# =========================================
vol_L_nominal = (L_base * W_base * T_base) * 1e-6
avg_mold_temp = (temp_fixed + temp_mobile) / 2
deltaT = abs(temp_fixed - temp_mobile)

# Packing (llenado) → densidad efectiva
pack_gain = 0.02*(fill_time1 - 3) + 0.015*(fill_time2 - 2)
pack_gain = float(np.clip(pack_gain, -0.05, 0.08))

# Densidad base del bead (si hay modelo, infiere; si no, 35→20 g/L por defecto)
bead_from_excel = None
if df_models is not None and sel:
    bead_from_excel = infer_bead_from_text(str(df_models.loc[idx].get("bead_text","")))
bead = bead_from_excel if bead_from_excel in BEAD_DENSITIES else 35
densidad_bead_nom = BEAD_DENSITIES[bead]
densidad_efectiva = densidad_bead_nom * (1.0 + pack_gain)

# Expansión
sigmoid = lambda x: 1/(1+np.exp(-x))
exp_steam = 0.03 * sigmoid(6*(steam_pressure - 1.35))
exp_time  = 0.015 * np.tanh((steam_time - 6)/6)
exp_temp  = 0.012 * np.tanh((avg_mold_temp - 55)/25)
expansion_total = 1.0 + exp_steam + exp_time + exp_temp

# Shrink / colapso
base_shrink = 0.008
shrink_deltaT = 0.001 * max(deltaT - 20, 0)
shrink_cooling = 0.012 * max(15 - cooling_time, 0)/15
shrink_oversteam = 0.008 if (steam_pressure > 1.8 and steam_time > 10) else 0.0
aging_relief = 0.6 * aging_quality
total_shrink = max((base_shrink + shrink_deltaT + shrink_cooling + shrink_oversteam) - aging_relief*0.008, 0)

mult_dim = expansion_total * (1.0 - total_shrink)
L = round(max(L_base * mult_dim, 0), 2)
W = round(max(W_base * mult_dim, 0), 2)
vol_L_final = (L * W * T_base) * 1e-6

# Pesos simulados
peso_wet_sim = densidad_efectiva * vol_L_final
# Humedad (depende de cooling/aging/agua/sobre-vapor)
hum_base = 0.025
hum_cooling = -0.015 * min(max((cooling_time-15)/25, 0), 1)
hum_aging   = -0.010 * aging_quality
hum_water   = -0.003 if water_on else 0
hum_penalty =  0.006 if (steam_pressure>1.8 and steam_time>10) else 0
hum_frac = np.clip(hum_base + hum_cooling + hum_aging + hum_water + hum_penalty, 0.005, 0.03)
peso_dry_sim = peso_wet_sim * (1 - hum_frac)

peso_wet_sim = round(peso_wet_sim, 1)
peso_dry_sim = round(peso_dry_sim, 1)

# =========================================
# DATOS TEÓRICOS (desde Excel) + REALES
# =========================================
st.subheader("📦 Modelo seleccionado")
if df_models is not None and sel:
    row = df_models.loc[idx]

    part_number = str(row.get("part_number",""))
    part_name   = str(row.get("part_name",""))
    bead_text   = str(row.get("bead_text",""))
    cure_time   = str(row.get("cure_time",""))

    wet_min = row.get("wet_min", None)
    wet_nom = row.get("wet_nom", None)
    wet_max = row.get("wet_max", None)
    dry_min = row.get("dry_min", None)
    dry_nom = row.get("dry_nom", None)
    dry_max = row.get("dry_max", None)

    c1, c2, c3, c4 = st.columns([1.2,1.6,1,1])
    c1.metric("Part Number", part_number)
    c2.metric("Part Name", part_name)
    c3.metric("Bead", bead_text if bead_text else f"{bead} (≈{densidad_bead_nom} g/L)")
    c4.metric("Cure Time", cure_time)

    st.subheader("📊 Resultados simulados")
    k1, k2, k3, k4 = st.columns(4)
    k1.metric("Wet SIM (g)", f"{peso_wet_sim:,.1f}")
    k2.metric("Dry SIM (g)", f"{peso_dry_sim:,.1f}")
    k3.metric("Largo SIM (mm)", f"{L:,.2f}")
    k4.metric("Ancho SIM (mm)", f"{W:,.2f}")

    st.subheader("🧪 Datos reales (ingrésalos para comparar)")
    colR1, colR2 = st.columns(2)
    with colR1:
        wet_real = st.number_input("Wet weight REAL (g)", min_value=0.0, value=0.0, step=1.0, format="%.1f")
    with colR2:
        dry_real = st.number_input("Dry weight REAL (g)", min_value=0.0, value=0.0, step=1.0, format="%.1f")

    # Comparaciones
    def estado(valor, vmin, vmax):
        if valor is None: return "—"
        if vmin is not None and valor < vmin: return "⬇️ Bajo"
        if vmax is not None and valor > vmax: return "⬆️ Alto"
        return "✅ OK"

    st.subheader("🧮 Comparativa Wet/Dry")
    rows = []
    rows.append({"Métrica":"Wet SIM vs Wet NOM", "Δ (g)": None if pd.isna(wet_nom) else round(peso_wet_sim - float(wet_nom),1),
                 "Estado": estado(peso_wet_sim, None if pd.isna(wet_min) else float(wet_min),
                                               None if pd.isna(wet_max) else float(wet_max))})
    rows.append({"Métrica":"Dry SIM vs Dry NOM", "Δ (g)": None if pd.isna(dry_nom) else round(peso_dry_sim - float(dry_nom),1),
                 "Estado": estado(peso_dry_sim, None if pd.isna(dry_min) else float(dry_min),
                                               None if pd.isna(dry_max) else float(dry_max))})
    if wet_real and wet_real>0:
        rows.append({"Métrica":"Wet REAL vs Wet NOM", "Δ (g)": None if pd.isna(wet_nom) else round(wet_real - float(wet_nom),1),
                     "Estado": estado(wet_real, None if pd.isna(wet_min) else float(wet_min),
                                               None if pd.isna(wet_max) else float(wet_max))})
    if dry_real and dry_real>0:
        rows.append({"Métrica":"Dry REAL vs Dry NOM", "Δ (g)": None if pd.isna(dry_nom) else round(dry_real - float(dry_nom),1),
                     "Estado": estado(dry_real, None if pd.isna(dry_min) else float(dry_min),
                                               None if pd.isna(dry_max) else float(dry_max))})

    df_cmp = pd.DataFrame(rows)
    st.dataframe(df_cmp, use_container_width=True)

    # Barras comparativas
    def barras(titulo, sim, real, nom):
        labels, vals = [], []
        if sim is not None: labels.append("SIM");  vals.append(sim)
        if real and real>0: labels.append("REAL"); vals.append(real)
        if pd.notna(nom):   labels.append("NOM");  vals.append(float(nom))
        if not labels:
            st.info("No hay datos suficientes para graficar."); return
        fig, ax = plt.subplots()
        ax.bar(labels, vals)
        ax.set_title(titulo); ax.set_ylabel("g")
        for i,v in enumerate(vals):
            ax.text(i, v, f"{v:.1f}", ha='center', va='bottom')
        st.pyplot(fig, use_container_width=True)

    cG1, cG2 = st.columns(2)
    with cG1:
        barras("WET: SIM vs REAL vs NOM", peso_wet_sim, wet_real, wet_nom)
    with cG2:
        barras("DRY: SIM vs REAL vs NOM", peso_dry_sim, dry_real, dry_nom)

    # Vista superior
    st.subheader("🧭 Vista superior de la pieza (W × L)")
    fig, ax = plt.subplots()
    rect = plt.Rectangle((0, 0), W, L, fc="#9AD1F5", ec="black")
    ax.add_patch(rect)
    ax.set_xlim(0, max(2000, W*1.1)); ax.set_ylim(0, max(2000, L*1.1))
    ax.set_aspect('equal'); ax.set_xlabel("Ancho (mm)"); ax.set_ylabel("Largo (mm)")
    st.pyplot(fig, use_container_width=True)

    # Exportar
    st.subheader("📥 Exportar reporte")
    out = {
        "Part Number":[part_number], "Part Name":[part_name], "Bead (texto)":[bead_text],
        "Cure Time":[cure_time], "Bead asignado":[bead], "Densidad bead (g/L)":[densidad_bead_nom],
        "Largo_nom_mm":[L_base], "Ancho_nom_mm":[W_base], "Espesor_mm":[T_base],
        "ICP_bar":[steam_pressure], "Vapor_s":[steam_time],
        "Temp_FIXED_C":[temp_fixed], "Temp_MOBILE_C":[temp_mobile], "DeltaT_C":[deltaT],
        "Fill1_s":[fill_time1], "Fill2_s":[fill_time2], "Cooling_s":[cooling_time],
        "Agua_ON":[water_on], "Aging_0a1":[aging_quality],
        "Wet_SIM_g":[peso_wet_sim], "Dry_SIM_g":[peso_dry_sim],
        "Wet_NOM_g":[None if pd.isna(wet_nom) else float(wet_nom)],
        "Dry_NOM_g":[None if pd.isna(dry_nom) else float(dry_nom)],
        "Wet_MIN_g":[None if pd.isna(wet_min) else float(wet_min)],
        "Wet_MAX_g":[None if pd.isna(wet_max) else float(wet_max)],
        "Dry_MIN_g":[None if pd.isna(dry_min) else float(dry_min)],
        "Dry_MAX_g":[None if pd.isna(dry_max) else float(dry_max)],
        "Wet_REAL_g":[wet_real if wet_real>0 else None],
        "Dry_REAL_g":[dry_real if dry_real>0 else None],
    }
    df_out = pd.DataFrame(out)
    st.dataframe(df_out, use_container_width=True)

    # Descarga directa
    buf = BytesIO()
    with pd.ExcelWriter(buf, engine="openpyxl") as wr:
        df_out.to_excel(wr, index=False, sheet_name="Reporte")
    buf.seek(0)
    st.download_button(
        "Descargar reporte (Excel)",
        data=buf,
        file_name="reporte_simulador_epp.xlsx",
        mime="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
    )

    # Guardar en Google Drive (opcional)
    st.sidebar.subheader("💾 Guardar en Drive (opcional)")
    save_to_drive = st.sidebar.checkbox("Guardar reporte en Drive")
    default_drive_path = "/content/drive/MyDrive/reporte_simulador_epp.xlsx"
    drive_path = st.sidebar.text_input("Ruta en Drive", value=default_drive_path)
    if save_to_drive and st.sidebar.button("Guardar ahora"):
        try:
            df_out.to_excel(drive_path, index=False, engine="openpyxl")
            st.sidebar.success(f"Guardado en: {drive_path}")
        except Exception as e:
            st.sidebar.error(f"Error al guardar: {e}")
else:
    st.warning("Selecciona un modelo para ver resultados.")

st.caption("Modelo heurístico educativo. Ajusta coeficientes con tus datos de línea para mayor precisión.")

# 3) Lanzar Streamlit en segundo plano y abrir túnel público
import threading, time, os
def run_streamlit():
    os.system("streamlit run app.py --server.port 6006")

threading.Thread(target=run_streamlit, daemon=True).start()
time.sleep(3)
print("Iniciando túnel público... (si la URL no carga, vuelve a ejecutar esta celda)")
# Inicia LocalTunnel y muestra la URL accesible
!lt --port 6006 --print-requests


Writing streamlit_app.py


In [5]:
!pip install streamlit pyngrok xlsxwriter matplotlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 5.6 MB/s eta 0:00:00


In [ ]:
!streamlit run streamlit_app.py & npx localtunnel --port 8501


⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙

⠹⠸⠼⠴⠦⠧⠇⠏
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.57.180.107:8501

your url is: https://late-melons-happen.loca.lt
